In [1]:
import pandas as pd
import numpy as np
import os
from itertools import chain
import csv
from collections import defaultdict
import plotly.express as px
import glob
from pathlib import Path

### Read in databases for DR, repeats, NE genes and lineages

In [2]:
# drug resistance positions
DB_DR = []
df = pd.read_csv('bin/db/drdb.txt',sep='\t')
df = df[df['Reference_Position'] != '-']
for snp in df['Reference_Position']:
    snp = snp.split('/')
    for i in snp:
        DB_DR.append(float(i))
        
# repeat positions and ne genes
df1 = pd.read_csv('bin/db/repeats.txt',sep='\t')
df2 = pd.read_csv('bin/db/ne_genes.txt',sep='\t')
df = pd.concat([df1,df2])
DB_REP_NE = df.apply(lambda row:list(range(row['Start'],row['End']+1)),axis=1).explode().astype(int).tolist()

# lineages
df_lin = pd.read_csv('bin/db/lineages.bed',sep='\t',index_col=1)
DB_LIN = df_lin.drop(columns=['Chromosome','End'])
colors = px.colors.qualitative.Dark24
colors.append(px.colors.qualitative.Light24[1])
colors = dict(zip(DB_LIN['Lineage_name'].unique(),colors))
DB_LIN['Lineage_color'] = DB_LIN['Lineage_name'].map(colors)

### Set global variables and read in files

In [4]:
# SET GLOBAL VARIABLES HERE
FREQ = 90
COV = 10
QUAL = 20
FRB = 0

OUTPUT = 'output/'
INPUT = 'input/'

In [5]:
# Rename files
directory = Path(INPUT+"snp/")
files = [f for f in directory.iterdir() if f.is_file()]
for old_path in files:
    dir_name = os.path.dirname(old_path)
    base_name = os.path.basename(old_path)
    if os.path.splitext(base_name)[1] == '.csv':
        new_base = base_name.split('_')[0] + '.snp'
        new_path = os.path.join(dir_name, new_base)
        os.rename(old_path, new_path)
    
directory = Path(INPUT+"cov/")
files = [f for f in directory.iterdir() if f.is_file()]
for old_path in files:
    dir_name = os.path.dirname(old_path)
    base_name = os.path.basename(old_path)
    if os.path.splitext(base_name)[1] == '.csv':
        new_base = base_name.split('_')[0] + '.cov'
        new_path = os.path.join(dir_name, new_base)
        os.rename(old_path, new_path)
    
directory = Path(INPUT+"stat/")
files = [f for f in directory.iterdir() if f.is_file()]
for old_path in files:
    dir_name = os.path.dirname(old_path)
    base_name = os.path.basename(old_path)
    if os.path.splitext(base_name)[1] == '.csv':
        new_base = base_name.split('_')[0] + '.stat'
        new_path = os.path.join(dir_name, new_base)
        os.rename(old_path, new_path)

In [6]:
snp_files = os.listdir(INPUT+'snp/')
cov_files = os.listdir(INPUT+'cov/')
stat_files = os.listdir(INPUT+'stat/')

### Generate a reference dictionary for all SNPs that passed QC

In [7]:
# TO DO:

# --> average coverage check using cov files before hand (in snp2tree this is pass_cov_check) 

In [8]:
# this will happen after QC of filenames etc and just the final files in the input

ref_dict = dict()
snp_dfs = dict()
snp_dfs_filtered = dict()

for file in snp_files:
    print(file)
    sample = file.replace('.snp','')
    df = pd.read_csv('input/snp/'+file)
    snp_dfs[sample] = df
    
    df['Sample'] = sample
    
    # check if SNV and not other variant
    df = df[df['Type'] == 'SNV']
    
    # normal QC
    df = df[(df['Frequency'] >= FREQ) & (df['Coverage'] >= COV) & (df['Average quality'] >= QUAL)]
    
    # forward reverse more than cut-off (keep to 0 unless for specific reasons)
    df = df[(df['Forward/reverse balance'] > FRB)]
    
    # exclude resistant genes
    df = df[~df['Reference Position'].isin(DB_DR)]
    
    # exclude repeats
    df = df[~df['Reference Position'].isin(DB_REP_NE)]
    
    # exclude exons
    df = df[df['Overlapping annotations'].notna()]
    df = df[~df['Amino acid change'].notna()]
    
    # exlude proximity snps
    arr_sorted = np.sort(df['Reference Position'])
    diff_next = np.diff(arr_sorted)
    too_close = diff_next <= 50
    mask = np.ones_like(arr_sorted, dtype=bool)
    mask[:-1] &= ~too_close
    mask[1:]  &= ~too_close
    df = df[df['Reference Position'].isin(arr_sorted[mask])]
    
    # add all reference alleles to ref_dict
    for index,row in df.iterrows():
        ref_dict[row['Reference Position']] = row['Reference']
    snp_dfs_filtered[sample] = df

# order dictionary
ref_dict = dict(sorted(ref_dict.items()))

TC00760619_S1.snp
TC00856554_S51.snp
TC00764189.snp
TC00831960_S32.snp
TC00855201_S148.snp
TC00862935_S189.snp
TC00836558_1.snp
TC00777418_S47.snp
TC00832902_S78.snp
TC00780707_S93.snp
TC00838386_S26.snp


### Create fasta file used for alignment and phylogenetic analysis

In [105]:
ffasta = open(OUTPUT+'snps.fasta','w')

# create reference for tree root
ffasta.write('>MTB\n'+''.join(ref_dict.values())+'\n')

# read in from filtered dataframes
for sample in snp_dfs_filtered:
    df = snp_dfs_filtered[sample]
    sample_dict = dict(zip(df['Reference Position'],df['Allele']))
    missing_keys = set(ref_dict.keys()) - set(sample_dict.keys())
    for key in missing_keys:
        sample_dict[key] = ref_dict[key]
    sample_dict = dict(sorted(sample_dict.items()))
    seq = ''.join(sample_dict.values())
    ffasta.write('>'+sample+'\n'+str(seq)+'\n')
ffasta.close()

### Assign lineages using unfiltered dataframes

In [139]:
# Variables

LIN_FREQ = 20
d_lin_name_number = dict(zip(DB_LIN['Lineage_number'],DB_LIN['Lineage_name']))

# create a lineage matrix that can later be used for viz if needed
lin_mat = pd.DataFrame()
for sample in snp_dfs:
    df = snp_dfs[sample]
    df = df.set_index('Reference Position')
    df = df.join(df_lin,rsuffix='_lin')
    df = df[df['Allele'] == df['Allele_lin']]
    df = df[df['Frequency'] >= LIN_FREQ]
    df = df.groupby('Lineage_number').mean('Fequency')
    for i in df.index:
        lin_mat.at[i,sample] = df.at[i,'Frequency']

# add lineage4.9 for samples that did not call any lineage
for sample in snp_dfs:
    if sample not in lin_mat.columns:
        lin_mat.at['lineage4.9',sample] = 100
        
df = lin_mat.reset_index()
df = df.rename(columns={'index':'Lineage Number'})
df['Lineage Name'] = df['Lineage Number'].map(d_lin_name_number)
df = pd.melt(df,id_vars=['Lineage Name','Lineage Number'])
df = df[df['value'].notna()]
df = df.rename(columns={'variable':'Sample','value':'Frequency'})
df = df[['Sample','Lineage Name','Lineage Number','Frequency']]
df.to_excel(OUTPUT+'/lineages.xlsx')
        
### BUILD FILE FOR LINEAGE ANNOTATION ON PHYLOGENETIC TREE ##

# create lineage dictionary from lineage matrix for iTOL tree       
lineage_tree = dict()
for col in lin_mat.columns:
    df = lin_mat[[col]].dropna()
    deep_lin = max(df.index.to_list(), key=len)
    lineage_tree[col] = deep_lin

# create dictionary for colors
colors = dict(zip(DB_LIN['Lineage_number'],DB_LIN['Lineage_color']))

# build iTOL file for lineage
flineage = open(OUTPUT+'lineage.txt','w')
flineage.write('DATASET_COLORSTRIP\nSEPARATOR TAB\nDATASET_LABEL\tLineage\nCOLOR\t#ff0000\n')
flineage.write('LEGEND_COLORS\t'+'\t'.join(colors.values())+'\n')
flineage.write('LEGEND_LABELS\t'+'\t'.join(colors.keys())+'\n')

flineage.write('\nDATA\n')
for sample in lineage_tree:
    flineage.write(sample+'\t'+colors[lineage_tree[sample]]+'\t'+lineage_tree[sample])
    flineage.write('\n')
flineage.close()

# RESISTANCE MODULE

In [6]:
os.system('mkdir '+OUTPUT+'Resistance')

mkdir: output/Resistance: File exists


256

In [7]:
WHORules = {'katG':   ['Isoniazid'],
            'pncA':   ['Pyrazinamide'],
            'ddn':    ['Pyrazinamide','Delamanid'],
            'fbiA':   ['Pyrazinamide','Delamanid'],
            'fbiB':   ['Pyrazinamide','Delamanid'],
            'fbiC':   ['Pyrazinamide','Delamanid'],
            'fgd1':   ['Pyrazinamide','Delamanid'],
            'Rv2983': ['Pyrazinamide','Delamanid'],
            'gid':    ['Streptomycin'],
            'ethA':   ['Prothionamide','Ethambutol'],
            'tlyA':   ['Capreomycin'],
            'pepQ' :  ['Bedaquiline','Clofazimine']}

In [8]:
# columns to select from files
SNP_COLUMNS = ['Type','Reference Position','Reference','Allele','Count','Coverage','Frequency',
               'Forward/reverse balance','Average quality','Coding region change','Amino acid change',
               'Overlapping annotations']
COV_COLUMNS = ['Name','Target region length','Mean coverage','Zero coverage bases']
STAT_COLUMNS = ['Mapped reads','Average coverage']

In [9]:
# parse databases
DR_WHO = pd.read_excel('../Resistance/WHO-UCN-TB-2023.5-eng.xlsx',skiprows=2)
DR_WHO = DR_WHO[['drug','gene','mutation','variant','tier',
                'Present_SOLO_SR','Present_SOLO_R','Present_SOLO_S','FINAL CONFIDENCE GRADING']]
DR_CTB = pd.read_excel('../Resistance/CTB_Mutation_Catalogue.xlsx')
DR_CTB.columns = ['CTB_variant','CTB_drug','CTB_FINAL_CONFIDENCE_GRADING']

/Users/k2259885/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/k2259885/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [10]:
# create dictionaries to use for resistotyping
dCTB = defaultdict(dict)
for index,row in DR_CTB.iterrows():
    dCTB[row['CTB_variant']][row['CTB_drug']] = row['CTB_FINAL_CONFIDENCE_GRADING']
dWHO = defaultdict(dict)
for index,row in DR_WHO.iterrows():
    dWHO[row['variant']][row['drug']] = row['FINAL CONFIDENCE GRADING']

dGene2Drug = defaultdict(list)
dDrug2Gene = defaultdict(list)

for index,row in DR_WHO.iterrows():
    dGene2Drug[row['gene']].append(row['drug'])
    dDrug2Gene[row['drug']].append(row['gene'])
for index,row in DR_CTB.iterrows():
    dGene2Drug[row['CTB_variant'].split('_')[0]].append(row['CTB_drug'])
    dDrug2Gene[row['CTB_drug']].append(row['CTB_variant'].split('_')[0])

# set lists
for gene in dGene2Drug:
    dGene2Drug[gene] = list(set(dGene2Drug[gene]))
for drug in dDrug2Gene:
    dDrug2Gene[drug] = list(set(dDrug2Gene[drug]))

In [11]:
# get all drug resistant genes
DR_GENES = list(DR_WHO['gene'].unique())
DR_GENES = list(set(DR_GENES + list(DR_CTB['CTB_variant'].str.split('_').str[0].unique())))

In [12]:
dDrug2Gene

defaultdict(list,
            {'Amikacin': ['whiB7',
              'rrs',
              'ccsA',
              'bacA',
              'Rv2477c',
              'whiB6',
              'eis'],
             'Bedaquiline': ['pepQ',
              'mtrA',
              'Rv0678',
              'lpqB',
              'Rv1979c',
              'mmpL5',
              'mmpS5',
              'mtrB',
              'atpE'],
             'Capreomycin': ['rrs',
              'ccsA',
              'bacA',
              'Rv2680',
              'whiB6',
              'rrl',
              'Rv2681',
              'tlyA'],
             'Clofazimine': ['fbiA',
              'pepQ',
              'fbiC',
              'Rv0678',
              'mmpS5',
              'Rv1979c',
              'mmpL5',
              'Rv2983',
              'fgd1',
              'fbiB'],
             'Delamanid': ['fbiA',
              'fbiC',
              'ddn',
              'Rv2983',
              'fgd1',
              'ndh',
      

In [13]:
dGene2Drug

defaultdict(list,
            {'bacA': ['Kanamycin', 'Streptomycin', 'Capreomycin', 'Amikacin'],
             'ccsA': ['Kanamycin', 'Capreomycin', 'Amikacin'],
             'eis': ['Kanamycin', 'Ethambutol', 'Amikacin'],
             'rrs': ['Kanamycin',
              'Streptomycin',
              'Capreomycin',
              'Amikacin',
              'Aminoglycoside'],
             'Rv2477c': ['Kanamycin',
              'Rifampicin',
              'Levofloxacin',
              'Streptomycin',
              'Moxifloxacin',
              'Ethambutol',
              'Amikacin'],
             'whiB6': ['Kanamycin', 'Capreomycin', 'Amikacin'],
             'whiB7': ['Kanamycin', 'Streptomycin', 'Amikacin'],
             'atpE': ['Bedaquiline'],
             'lpqB': ['Bedaquiline', 'Rifampicin'],
             'mmpL5': ['Bedaquiline', 'Clofazimine'],
             'mmpS5': ['Bedaquiline', 'Clofazimine'],
             'mtrA': ['Bedaquiline', 'Rifampicin'],
             'mtrB': ['Bedaquiline', 

In [149]:
# merge into single dataframe

# to do --> any quality control to be included in resistotyping
# to do --> inhA promoter issue

DR_QUAL = 20
DR_FRB = 0
DR_FREQ = 5
DR_COV = 5

dRes = defaultdict(lambda: defaultdict(list))

columns = []
for drug in dDrug2Gene:
    columns.append( ('Resistotype',drug,'') )
    
queries = ['Coding region change','Amino acid change','Resistance level']
for drug in dDrug2Gene:
    for gene in dDrug2Gene[drug]:
        for query in queries:
            columns.append( (drug,gene,query) )
            
columns = pd.MultiIndex.from_tuples(columns)
mainOut = pd.DataFrame(columns=columns)

for file in snp_files:
    indexes = []
    sample = file.replace('.snp','')
    dfResSample = pd.DataFrame(columns=['Drug','Gene','Coding region change','Amino acid change','Level'])
    counter = 0
    
    # read in files and do QC 
    snp = pd.read_csv('input/snp/'+sample+'.snp')[SNP_COLUMNS]
    snp = snp[snp['Average quality'] >= DR_QUAL]
    snp = snp[snp['Forward/reverse balance'] > DR_FRB]
    snp = snp[snp['Frequency'] >= DR_FREQ]
    snp = snp[snp['Coverage'] >= DR_COV]
    
    cov = pd.read_csv('input/cov/'+sample+'.cov')[COV_COLUMNS]
    stat = pd.read_csv('input/stat/'+sample+'.stat')[STAT_COLUMNS]
    
    # split column to get gene
    df = snp.copy()
    df = df[df['Overlapping annotations'].notna()]
    df = df[df['Overlapping annotations'].str.contains('Gene')]
    df['Gene'] = df['Overlapping annotations'].str.split('Gene: ').str[1].str.split().str[0]
    
    # get only DR genes
    df = df[df['Gene'].isin(DR_GENES)]
    # merge snp and cov
    df = pd.merge(df, cov, left_on='Gene', right_on='Name')
    df = df.drop(columns=['Gene'])
    # merge with stat
    for col in stat.columns:
        df[col] = stat.at[0,col]
    
    # add sample name as a column and move it to first column
    df['Sample'] = sample
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
    
    # create resistance dictionary
    
    # check for ungraded mutations -  add ungraded for all genes associated with resistance
    for index,row in df.iterrows():
        if row['Name'] in dGene2Drug:
            for drug in dGene2Drug[row['Name']]:
                dfResSample.loc[len(dfResSample)] = [drug,str(row['Name']),str(row['Coding region change']),
                                                     str(row['Amino acid change']),'Ungraded']
                if 'Ungraded' not in dRes[sample][drug]:
                    dRes[sample][drug].append('Ungraded')
                    indexes.append(index)
                 
    # for ctb
    df['tmp1'] = df['Name']+'_'+df['Coding region change'].str.split('c.').str[1] 
    df['tmp2'] = df['Name']+'_'+df['Amino acid change'].str.split('p.').str[1]
    for index,row in df.iterrows():
        if row['tmp1'] in dCTB:
            for drug in dCTB[row['tmp1']]:
                dRes[sample][drug].append(dCTB[row['tmp1']][drug])
                indexes.append(index)
                dfResSample.loc[len(dfResSample)] = [drug,row['Name'],str(row['tmp1']),
                                                     str(row['tmp2']),dCTB[row['tmp1']][drug]]
        if row['tmp2'] in dCTB:
            for drug in dCTB[row['tmp2']]:
                dRes[sample][drug].append(dCTB[row['tmp2']][drug])
                indexes.append(index)
                dfResSample.loc[len(dfResSample)] = [drug,row['Name'],str(row['tmp1']),
                                                     str(row['tmp2']),dCTB[row['tmp2']][drug]]
    df = df.drop(columns=['tmp1','tmp2'])
    
    # for WHO
    df['tmp1'] = df['Name']+'_'+df['Coding region change'].str.split(':').str[1] 
    df['tmp2'] = df['Name']+'_'+df['Amino acid change'].str.split(':').str[1]
    for index,row in df.iterrows():
        if row['tmp1'] in dWHO:
            for drug in dWHO[row['tmp1']]:
                dRes[sample][drug].append(dWHO[row['tmp1']][drug])
                indexes.append(index)
                dfResSample.loc[len(dfResSample)] = [drug,row['Name'],str(row['tmp1']),
                                                     str(row['tmp2']),dWHO[row['tmp1']][drug]]
        if row['tmp2'] in dWHO:
            for drug in dWHO[row['tmp2']]:
                dRes[sample][drug].append(dWHO[row['tmp2']][drug])
                indexes.append(index)
                dfResSample.loc[len(dfResSample)] = [drug,row['Name'],str(row['tmp1']),
                                                     str(row['tmp2']),dWHO[row['tmp2']][drug]]
    df = df.drop(columns=['tmp1','tmp2'])
    
    # for WHO expert
    df2 = df[(df['Type'].isin(['Insertion','Deletion'])) | 
             (df['Amino acid change'].str.contains('*',na=False,regex=False))]
    for index,row in df2.iterrows():
        if row['Name'] in WHORules:
            for drug in WHORules[row['Name']]:
                dRes[sample][drug].append('Expert_R')
                dfResSample.loc[len(dfResSample)] = [drug,row['Name'],str(row['Coding region change']),
                                                     str(row['Amino acid change']),'Expert_R']
    if 'Rv0678' in list(df2['Name']):
        if 'mmpL5' in list(df2['Name']):
            pass
        else:
            dRes[sample]['Bedaquiline'].append('Expert_R')
            dRes[sample]['Clofazimine'].append('Expert_R')
            df3 = df2[df2['Name'] == 'Rv0678']
            for index,row in df3.iterrows():
                dfResSample.loc[len(dfResSample)] = ['Bedaquiline',row['Name'],
                                                     str(row['Coding region change']),
                                                     str(row['Amino acid change']),
                                                     'Expert_R']
                dfResSample.loc[len(dfResSample)] = ['Clofazimine',row['Name'],
                                                     str(row['Coding region change']),
                                                     str(row['Amino acid change']),
                                                     'Expert_R']
            
    df = df.loc[list(set(indexes))]
    mainOut = resistotyping(dfResSample,mainOut,sample)
    
dfRes = buildResistogram(dRes)
for i in dfRes.index:
    for c in dfRes.columns:
        mainOut.at[i,('Resistotype',c,'')] = dfRes.at[i,c]
mainOut = mainOut.dropna(axis=1, how='all')

In [150]:
mainOut.to_excel('test.xlsx')

In [148]:
# this is to have the resistogram also alone
def buildResistogram(d):
    dfRes = pd.DataFrame()
    for sample in d:
        for drug in d[sample]:
            if '1) Assoc w R' in d[sample][drug]:
                #dfRes.at[sample,drug] = '1'
                dfRes.at[sample,drug] = 'AWR'
            elif '2) Assoc w R - Interim' in d[sample][drug]:
                #dfRes.at[sample,drug] = '2'
                dfRes.at[sample,drug] = 'AWR-I'
            elif 'Expert_R' in d[sample][drug]:
                #dfRes.at[sample,drug] = '3'
                dfRes.at[sample,drug] = 'WHO-E'
            elif '3) Uncertain significance' in d[sample][drug]:
                #dfRes.at[sample,drug] = '4'
                dfRes.at[sample,drug] = 'US'
            elif '5) Not assoc w R' in d[sample][drug]:
                #dfRes.at[sample,drug] = '6'
                dfRes.at[sample,drug] = 'NAWR'
            elif '4) Not assoc w R - Interim' in d[sample][drug]:
                #dfRes.at[sample,drug] = '7'
                dfRes.at[sample,drug] = 'NAWR-I'
            elif 'R' in d[sample][drug]:
                #dfRes.at[sample,drug] = '5'
                dfRes.at[sample,drug] = 'CTB-R'
            elif 'Ungraded' in d[sample][drug]:
                #dfRes.at[sample,drug] = 'Ungraded'
                dfRes.at[sample,drug] = 'UNGR'
    dfRes = dfRes.fillna('WT')
    return dfRes

In [145]:
def resistotyping(df,outdf,sample):
    for drug in df['Drug'].unique():
        for gene in df['Gene'].unique():
            df2 = df[(df['Drug'] == drug) & (df['Gene'] == gene)]
            if '1) Assoc w R' in df2['Level']:
                df2 = df2[df2['Level'] == '1) Assoc w R']
            elif '2) Assoc w R - Interim' in df2['Level']:
                df2 = df2[df2['Level'] == '2) Assoc w R - Interim']
            elif 'Expert_R' in df2['Level']:
                df2 = df2[df2['Level'] == 'Expert_R']
            elif '3) Uncertain significance' in df2['Level']:
                df2 = df2[df2['Level'] == '3) Uncertain significance']
            elif '5) Not assoc w R' in df2['Level']:
                df2 = df2[df2['Level'] == '5) Not assoc w R']
            elif '4) Not assoc w R - Interim' in df2['Level']:
                df2 = df2[df2['Level'] == '4) Not assoc w R - Interim']
            elif 'R' in df2['Level']:
                df2 = df2[df2['Level'] == 'R']
            elif 'Ungraded' in df2['Level']:
                df2 = df2[df2['Level'] == 'Ungraded']
            
            nucl_change = str(' / '.join(df2['Coding region change']))
            aa_change = str(' / '.join(df2['Amino acid change']))
    
            if len(df2.index) > 0:
                level = list(set(df2['Level']))[0]
                # Ungraded will have several mutations (can remove if don't want all the mutations in there)
                if level == 'Ungraded': 
                    outdf.at[sample,(drug,gene,'Coding region change')] = nucl_change
                    outdf.at[sample,(drug,gene,'Amino acid change')] = aa_change
                    outdf.at[sample,(drug,gene,'Resistance level')] = level
                else:
                    outdf.at[sample,(drug,gene,'Coding region change')] = nucl_change
                    outdf.at[sample,(drug,gene,'Amino acid change')] = aa_change
                    outdf.at[sample,(drug,gene,'Resistance level')] = level
            
    return outdf       

Resistotype                                                 
                   Amikacin Bedaquiline Capreomycin Clofazimine Delamanid   
                                                                            
TC00760619_S1           NaN         NaN         NaN         NaN       NaN  \
TC00856554_S51          NaN         NaN         NaN         NaN       NaN   
TC00764189              NaN         NaN         NaN         NaN       NaN   
TC00831960_S32          NaN         NaN         NaN         NaN       NaN   
TC00855201_S148         NaN         NaN         NaN         NaN       NaN   
TC00862935_S189         NaN         NaN         NaN         NaN       NaN   
TC00836558_1            NaN         NaN         NaN         NaN       NaN   
TC00777418_S47          NaN         NaN         NaN         NaN       NaN   
TC00832902_S78          NaN         NaN         NaN         NaN       NaN   
TC00780707_S93          NaN         NaN         NaN         NaN       NaN   
TC00838386_S26          NaN         NaN         NaN         NaN       NaN   

                                                                         ...   
                Ethambutol Ethionamide Isoniazid Kanamycin Levofloxacin  ...   
                                                                         ...   
TC00760619_S1          NaN         NaN       NaN       NaN          NaN  ...  \
TC00856554_S51         NaN         NaN       NaN       NaN          NaN  ...   
TC00764189             NaN         NaN       NaN       NaN          NaN  ...   
TC00831960_S32         NaN         NaN       NaN       NaN          NaN  ...   
TC00855201_S148        NaN         NaN       NaN       NaN          NaN  ...   
TC00862935_S189        NaN         NaN       NaN       NaN          NaN  ...   
TC00836558_1           NaN         NaN       NaN       NaN          NaN  ...   
TC00777418_S47         NaN         NaN       NaN       NaN          NaN  ...   
TC00832902_S78         NaN         NaN       NaN       NaN          NaN  ...   
TC00780707_S93         NaN         NaN       NaN       NaN          NaN  ...   
TC00838386_S26         NaN         NaN       NaN       NaN          NaN  ...   

                     Fluoroquinolone                                      
                                gyrB                                      
                Coding region change Amino acid change Resistance level   
TC00760619_S1                    NaN               NaN              NaN  \
TC00856554_S51                   NaN               NaN              NaN   
TC00764189                       NaN               NaN              NaN   
TC00831960_S32                   NaN               NaN              NaN   
TC00855201_S148                  NaN               NaN              NaN   
TC00862935_S189                  NaN               NaN              NaN   
TC00836558_1                     NaN               NaN              NaN   
TC00777418_S47                   NaN               NaN              NaN   
TC00832902_S78                   NaN               NaN              NaN   
TC00780707_S93                   NaN               NaN              NaN   
TC00838386_S26                   NaN               NaN              NaN   

                         Ethamobutol                                      
                                manB                                      
                Coding region change Amino acid change Resistance level   
TC00760619_S1                    NaN               NaN              NaN  \
TC00856554_S51                   nan               NaN         Ungraded   
TC00764189                       NaN               NaN              NaN   
TC00831960_S32                   nan               NaN         Ungraded   
TC00855201_S148                  NaN               NaN              NaN   
TC00862935_S189                  NaN               NaN              NaN   
TC00836558_1                     NaN               NaN              NaN   
TC00777418_S47 

In [40]:
dfRes

,Isoniazid,Levofloxacin,Moxifloxacin,Ethambutol,Clofazimine,Delamanid,Capreomycin,Amikacin,Kanamycin,Ethionamide,Rifamycin,Rifampicin,Bedaquiline,Streptomycin,Aminoglycoside,Pyrazinamide,Linezolid,Fluoroquinolone,Ethamobutol
TC00760619_S1,1.0,1.0,1.0,1.0,4.0,7.0,6.0,4.0,4.0,4.0,5.0,1.0,4.0,4.0,Ungraded,1.0,6.0,WT,WT
TC00856554_S51,4.0,6.0,6.0,1.0,4.0,7.0,6.0,6.0,4.0,4.0,5.0,1.0,4.0,6.0,Ungraded,4.0,6.0,Ungraded,Ungraded
TC00764189,1.0,1.0,1.0,1.0,2.0,7.0,6.0,Ungraded,Ungraded,4.0,5.0,1.0,2.0,1.0,Ungraded,1.0,6.0,WT,WT
TC00831960_S32,4.0,1.0,1.0,1.0,6.0,7.0,6.0,6,4,4.0,5.0,1.0,4.0,1.0,Ungraded,1.0,6.0,Ungraded,Ungraded
TC00855201_S148,1.0,1.0,1.0,1.0,4.0,7.0,6.0,Ungraded,Ungraded,4.0,Ungraded,1.0,4.0,4.0,Ungraded,1.0,6.0,WT,WT
TC00862935_S189,1.0,4.0,4.0,1.0,4.0,7.0,6.0,Ungraded,Ungraded,4.0,Ungraded,1.0,4.0,4.0,Ungraded,1.0,6.0,WT,Ungraded
TC00836558_1,1.0,4.0,4.0,1.0,2.0,7.0,6.0,4,4,4.0,5,1.0,1.0,6.0,Ungraded,4.0,6.0,WT,WT
TC00777418_S47,1.0,1.0,1.0,1.0,4.0,7.0,6.0,Ungraded,Ungraded,4.0,5,1.0,4.0,2.0,Ungraded,1.0,6.0,Ungraded,Ungraded
TC00832902_S78,1.0,4.0,4.0,1.0,4.0,7.0,6.0,Ungraded,Ungraded,4.0,Ungraded,1.0,4.0,4.0,Ungraded,1.0,6.0,Ungraded,Ungraded
TC00780707_S93,1.0,1.0,1.0,1.0,4.0,7.0,6.0,Ungraded,Ungraded,4.0,5,1.0,4.0,4.0,Ungraded,1.0,6.0,WT,WT


In [271]:
dfRes.to_excel('output/Resistance/resistogram.xlsx')

In [ ]:
    # perform WHO Expert ruling
#    df = WHOExpert(df)
    
#     # merge with CTB mutations
#     df_ctb = df.copy()
#     df_ctb['tmp1'] = df_ctb['Name']+'_'+df_ctb['Coding region change'].str.split('c.').str[1] 
#     df_ctb['tmp2'] = df_ctb['Name']+'_'+df_ctb['Amino acid change'].str.split('p.').str[1]
#     merged1 = pd.merge(df_ctb, DR_CTB, left_on='tmp1',right_on='CTB_variant',how='left',suffixes=('','_from_tmp1'))
#     tmp1_na_mask = merged1['CTB_variant'].isna()
#     merged2 = pd.merge(df_ctb[tmp1_na_mask],DR_CTB,left_on='tmp2',right_on='CTB_variant',how='left',suffixes=('', '_from_tmp2'))
#     df_ctb = pd.concat([merged1[~tmp1_na_mask], merged2], ignore_index=True)
#     df_ctb = df_ctb.drop(columns=['tmp1','tmp2'])
    
#     # merge with WHO mutations
#     df_who = df.copy()
#     df_who['tmp1'] = df_who['Name']+'_'+df_who['Coding region change'].str.split(':').str[1] 
#     df_who['tmp2'] = df_who['Name']+'_'+df_who['Amino acid change'].str.split(':').str[1]
#     merged1 = pd.merge(df_who, DR_WHO, left_on='tmp1',right_on='variant',how='left',suffixes=('','_from_tmp1'))
#     tmp1_na_mask = merged1['variant'].isna()
#     merged2 = pd.merge(df_who[tmp1_na_mask],DR_WHO,left_on='tmp2',right_on='variant',how='left',suffixes=('', '_from_tmp2'))
#     df_who = pd.concat([merged1[~tmp1_na_mask], merged2], ignore_index=True)
#     df_who = df_who.drop(columns=['tmp1','tmp2'])
    
#     # create single file for sample resistance and write to excel file
#     df = pd.concat([df_ctb,df_who])
#     # select variants with DR annotation in CTB or WHO
#     df = df[(df['CTB_FINAL_CONFIDENCE_GRADING'].notna()) | (df['FINAL CONFIDENCE GRADING'].notna())]
    
    #df.to_excel(OUTPUT+'Resistance/'+sample+'_RES.xlsx', index=False)
    
#     #resistotype = resistotyping(df)
#     #for drug in resistotype:
#     #    print(str(drug) + '\t' + str(resistotype[drug]))

In [93]:
snp

,Type,Reference Position,Reference,Allele,Count,Coverage,Frequency,Forward/reverse balance,Average quality,Coding region change,Amino acid change,Overlapping annotations
19,SNV,1849,C,A,182,182,100.000000,0.492891,33.670330,NaN,NaN,NaN
21,SNV,1977,A,G,157,158,99.367089,0.456522,33.707006,NaN,NaN,NaN
80,SNV,4013,T,C,137,137,100.000000,0.469512,33.182482,NP_214517.1:c.734T>C,NP_214517.1:p.Ile245Thr,"CDS: recF, Gene: recF"
99,SNV,7362,G,C,110,112,98.214286,0.393443,33.054545,NP_214520.1:c.61G>C,NP_214520.1:p.Glu21Gln,"CDS: gyrA, Gene: gyrA"
100,SNV,7570,C,T,139,139,100.000000,0.406250,32.604317,NP_214520.1:c.269C>T,NP_214520.1:p.Ala90Val,"CDS: gyrA, Gene: gyrA"
...,...,...,...,...,...,...,...,...,...,...,...,...
55658,SNV,4407588,T,C,179,179,100.000000,0.500000,33.039106,NP_218436.2:c.615A>G,NaN,"CDS: gid, Gene: gid"
55662,SNV,4407927,T,G,139,141,98.581560,0.494118,33.410072,NP_218436.2:c.276A>C,NP_218436.2:p.Glu92Asp,"CDS: gid, Gene: gid"
55663,SNV,4407967,A,G,144,144,100.000000,0.463158,33.013889,NP_218436.2:c.236T>C,NP_218436.2:p.Leu79Ser,"CDS: gid, Gene: gid"
55669,SNV,4408899,G,A,135,136,99.264706,0.496894,32.888889,NaN,NaN,NaN
